# BERT (mobile) Hotel Sentiment Distribution Shift Performance


Here we use the  `bert-
uncased_L-24_H-128_B-512_A-4_F-4` (mobile BERT)  pretrained model for investigating the distribution shift when we train on OpSpam hotel reviews with **positive** sentiment and test on reviews with **negative** sentiments.

In [1]:
!pip install bert-tensorflow==1.0.1
!pip install tensorflow==1.15.0

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 412.3MB 27kB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 512kB 35.0MB/s 
     |████████████████████████████████| 3.8MB 36.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=5e3f734a4be2cdabd28150aa1311b740d020edcd4dd4f94d523ffb3d1788b2ec
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Unin

In [2]:
# Making sure the environment is set up correctly for anyone running this notebook
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import run_classifier_with_tfhub
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np

**Getting the Data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tensorflow import keras
from bert import modeling
import os
import re

def download_and_load_dataset(force_download=True):
  return pd.read_csv("/content/drive/MyDrive/Colab Notebooks/deceptive-opinion.csv")

def refinereviewText(row):
    return row['reviewText'].lower()

reviews = download_and_load_dataset()
reviews = reviews[['deceptive','polarity','text']]
reviews.columns = ['deceptive','polarity','reviewText']
reviews['fakeLabel'] = np.where(reviews['deceptive'] == 'deceptive',1,0)
reviews['reviewText'] = reviews.apply(refinereviewText, axis=1)
reviews.head()

,deceptive,polarity,reviewText,fakeLabel
0,truthful,positive,we stayed for a one night getaway with family ...,0
1,truthful,positive,triple a rate with upgrade to view room was le...,0
2,truthful,positive,this comes a little late as i'm finally catchi...,0
3,truthful,positive,the omni chicago really delivers on all fronts...,0
4,truthful,positive,i asked for a high floor away from the elevato...,0


**Processing the data**

In [5]:
reviews_positive = reviews[reviews['polarity'] == 'positive']
reviews_negative = reviews[reviews['polarity'] == 'negative']

reviews_positive = reviews_positive[['reviewText','fakeLabel']]
reviews_negative = reviews_negative[['reviewText','fakeLabel']]

print("Number of positive reviews:", len(reviews_positive))
print("Number of negative reviews:", len(reviews_negative))

Number of positive reviews: 800
Number of negative reviews: 800


In [6]:
reviews_positive.head()

,reviewText,fakeLabel
0,we stayed for a one night getaway with family ...,0
1,triple a rate with upgrade to view room was le...,0
2,this comes a little late as i'm finally catchi...,0
3,the omni chicago really delivers on all fronts...,0
4,i asked for a high floor away from the elevato...,0


In [7]:
reviews_negative.head()

,reviewText,fakeLabel
800,booked a room w/ a queen bed for 2 nights for ...,0
801,i arrived and my friends were already there an...,0
802,this was the 2nd time that we have stayed at t...,0
803,my daughter and i woke in the morning wanting ...,0
804,they allowed parties to go on all night and bo...,0


Everything until this cell has been run of the mill data processing and setup. Now, we split OpSpam into 80/20 train/test, and define the labels we wish to classify.

In [8]:
from sklearn.utils import shuffle

inputs_positive = reviews_positive['reviewText']
labels_positive = [int(x) for x in reviews_positive['fakeLabel']]

inputs_negative = reviews_negative['reviewText']
labels_negative = [int(x) for x in reviews_negative['fakeLabel']]

# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [9]:
#OPTION 1 - TRAIN WITH POSITIVE, TEST WITH NEGATIVE
training_X, training_y = np.array(shuffle(inputs_positive,labels_positive))
test_X, test_y = np.array(shuffle(inputs_negative,labels_negative))

In [10]:
train = pd.DataFrame(zip(training_X, training_y), columns=["x", "y"])
test = pd.DataFrame(zip(test_X, test_y), columns=["x", "y"])
train.y.value_counts(), test.y.value_counts()

(1    400
 0    400
 Name: y, dtype: int64, 1    400
 0    400
 Name: y, dtype: int64)

In [11]:
train

,x,y
0,we just got back from 3 nights at the sofitel....,0
1,"i travel a lot for business and quite frankly,...",0
2,i recently stayed at the hotel monaco and i mu...,1
3,"very quaint and romantic, yet masculine. i've ...",0
4,we stayed here for a weekend trip to chicago a...,1
...,...,...
795,"we absolutely loved the knickerbocker. now, if...",0
796,ambassador east is an awesome hotel!!! underst...,0
797,this hotel was incredible! haven't had a bette...,1
798,my husband and i stayed here for three nights ...,0


The following cell creates placeholder models for the training and testing BERT inputs, and creates a model_specific tokenizer.

In [12]:
DATA_COLUMN = 'x'
LABEL_COLUMN = 'y'

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

BERT_MODEL_HUB = "https://tfhub.dev/google/mobilebert/uncased_L-24_H-128_B-512_A-4_F-4_OPT/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Now we specify how long we wish our reviews to be truncated to, and use our placeholder models created previously to turn our train and test sets into features usable by BERT.

In [13]:
# We'll set sequences to be at most 320 tokens long.
MAX_SEQ_LENGTH = 320
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 800


INFO:tensorflow:Writing example 0 of 800


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


**Model Definition**

This model is a modified version of the one used by Google for IMDB review classification (sentiment analysis), and is set up to suit our task (combined sentence classification rather than token-level output).

In [14]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

 
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


**Hyperparameter Tuning**

Here we define our hyperparams and use them to calculate the number of training and warmup steps. We also define how often we want to make model checkpoints.

In [15]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [16]:
print(num_train_steps, num_warmup_steps)

150 15


In [17]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir='/content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [18]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efe47f65a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efe47f65a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

**Training**

In [20]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 150 into /content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile/model.ckpt.


INFO:tensorflow:Saving checkpoints for 150 into /content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile/model.ckpt.


INFO:tensorflow:Loss for final step: 0.22539037.


INFO:tensorflow:Loss for final step: 0.22539037.


Training took time  0:11:14.730320


In [21]:
print(num_train_steps)

150


Now BERT has finished training, let's evaluate it on our testing set.

In [22]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [23]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 150: auc = 0.7737499, eval_accuracy = 0.77375, f1_score = 0.7425319, false_negatives = 139.0, false_positives = 42.0, global_step = 150, loss = 0.59283996, precision = 0.8613861, recall = 0.6525, true_negatives = 358.0, true_positives = 261.0


INFO:tensorflow:Saving dict for global step 150: auc = 0.7737499, eval_accuracy = 0.77375, f1_score = 0.7425319, false_negatives = 139.0, false_positives = 42.0, global_step = 150, loss = 0.59283996, precision = 0.8613861, recall = 0.6525, true_negatives = 358.0, true_positives = 261.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: /content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile/model.ckpt-150


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: /content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile/model.ckpt-150


{'auc': 0.7737499,
 'eval_accuracy': 0.77375,
 'f1_score': 0.7425319,
 'false_negatives': 139.0,
 'false_positives': 42.0,
 'global_step': 150,
 'loss': 0.59283996,
 'precision': 0.8613861,
 'recall': 0.6525,
 'true_negatives': 358.0,
 'true_positives': 261.0}

In [24]:
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
estimator.evaluate(input_fn=train_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 150: auc = 0.97749996, eval_accuracy = 0.9775, f1_score = 0.97715735, false_negatives = 15.0, false_positives = 3.0, global_step = 150, loss = 0.07742286, precision = 0.992268, recall = 0.9625, true_negatives = 397.0, true_positives = 385.0


INFO:tensorflow:Saving dict for global step 150: auc = 0.97749996, eval_accuracy = 0.9775, f1_score = 0.97715735, false_negatives = 15.0, false_positives = 3.0, global_step = 150, loss = 0.07742286, precision = 0.992268, recall = 0.9625, true_negatives = 397.0, true_positives = 385.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: /content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile/model.ckpt-150


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: /content/drive/MyDrive/Colab Notebooks/BERT_sentiment_mobile/model.ckpt-150


{'auc': 0.97749996,
 'eval_accuracy': 0.9775,
 'f1_score': 0.97715735,
 'false_negatives': 15.0,
 'false_positives': 3.0,
 'global_step': 150,
 'loss': 0.07742286,
 'precision': 0.992268,
 'recall': 0.9625,
 'true_negatives': 397.0,
 'true_positives': 385.0}